# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902040


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/32 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/32 [00:06<01:39,  3.31s/it]

Rendering models:   9%|▉         | 3/32 [00:07<01:15,  2.59s/it]

Rendering models:  12%|█▎        | 4/32 [00:07<00:53,  1.90s/it]

Rendering models:  16%|█▌        | 5/32 [00:09<00:48,  1.80s/it]

...

Rendering models:  91%|█████████ | 29/32 [00:27<00:02,  1.04it/s]

Rendering models:  94%|█████████▍| 30/32 [00:28<00:01,  1.02it/s]

Rendering models:  97%|█████████▋| 31/32 [00:28<00:00,  1.06it/s]

Rendering models: 100%|██████████| 32/32 [00:29<00:00,  1.08it/s]

equidad1                              0.043914
klmasters                             0.000544
kayleebug2017                         0.000883
not-logged-in-6ab0e00fa5a467f5f742    0.000936
Linda_J._Berkel                       0.000321
Drew_Farnsworth                       0.000365
Thedunkmasta                          0.002181
not-logged-in-1bfb5ea67e2d65ad84ff    0.058475
not-logged-in-e8b405843217e2525550    0.002397
Kamphuisjes                           0.000233
not-logged-in-6c3c85931d83a1164a0d    0.002120
sn346808                              0.018870
Arbitrationer                         0.004897
MrsDerps                              0.000854
KrisKeogh                             0.006385
engrg                                 0.000700
Frosst.matt                           0.000778
IrisR                                 0.030081
plankjames                            0.000279
not-logged-in-4bc570689341235fd74d    0.000336
ElisabethB                            0.000299
makainu      

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())